In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request
from os import listdir
from os.path import isfile, join, abspath
import pandas as pd
import xlwings as xw
import numpy as np
import quandl
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
from datetime import datetime, timedelta
import calendar
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from TFANN import ANNR
import seaborn as sns
from scipy import stats
import statistics
import os
%matplotlib inline 

C:\Users\clare\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Gets the currencies involved
dcc = pd.read_csv('datcommodcurr.csv',skiprows=1)#, header = None, sheet_name = 'Reference_Sheet')
dcc.index = dcc["Unnamed: 0"]
listcurr1 = []
for i in dcc.columns:
    if "/" in i:
        listcurr1.append(i)
ID = pd.read_excel("Investing.com ID.xlsx")
ID
for t in range(len(ID)):
    if ID["Currency"][t] == 'USD/ZAR':
        curr_id = ID["CurrID"][t]
        smlID = ID["smlID"][t]
        print(curr_id)
        print(smlID)
listcurr1

17
106698


['AUD/CAD',
 'AUD/CHF',
 'AUD/EUR',
 'AUD/GBP',
 'AUD/JPY',
 'AUD/NZD',
 'AUD/USD',
 'BTC/EUR',
 'CAD/AUD',
 'CAD/CHF',
 'CAD/EUR',
 'CAD/JPY',
 'CHF/AUD',
 'CHF/EUR',
 'CHF/GBP',
 'CHF/JPY',
 'EUR/AUD',
 'EUR/CAD',
 'EUR/DKK',
 'EUR/GBP',
 'EUR/JPY',
 'EUR/NOK',
 'EUR/NZD',
 'EUR/RUB',
 'EUR/SEK',
 'EUR/TRY',
 'EUR/USD',
 'EUR/ZAR',
 'GBP/AUD',
 'GBP/CAD',
 'GBP/CHF',
 'GBP/DKK',
 'GBP/EUR',
 'GBP/JPY',
 'GBP/NZD',
 'GBP/USD',
 'JPY/CHF',
 'JPY/CNY',
 'JPY/EUR',
 'JPY/GBP',
 'NOK/JPY',
 'NOK/SEK',
 'NZD/JPY',
 'NZD/USD',
 'RUB/GBP',
 'SEK/JPY',
 'USD/ARS',
 'USD/BRL',
 'USD/BTC',
 'USD/CAD',
 'USD/CHF',
 'USD/CNY',
 'USD/CZK',
 'USD/DKK',
 'USD/EGP',
 'USD/HKD',
 'USD/HUF',
 'USD/IDR',
 'USD/ILS',
 'USD/INR',
 'USD/JPY',
 'USD/KRW',
 'USD/MXN',
 'USD/MYR',
 'USD/NOK',
 'USD/PLN',
 'USD/RUB',
 'USD/SEK',
 'USD/SGD',
 'USD/THB',
 'USD/TRY',
 'USD/UAH',
 'USD/ZAR']

In [3]:
def investing(i,std,etd):
    ID = pd.read_excel("Investing.com ID.xlsx")
    for t in range(len(ID)):
        if ID["Currency"][t] == i:
            headers = {
                "Host": "www.investing.com",
                "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:67.0) Gecko/20100101 Firefox/67.0",
                "Accept": "text/plain, */*; q=0.01",
                "Accept-Language": "en-US,en;q=0.5",
                "Accept-Encoding": "gzip, deflate, br",
                "Referer": "https://www.investing.com/currencies/"+i.replace("/","-").lower()+"-historical-data",
                "Content-Type": "application/x-www-form-urlencoded",
                "X-Requested-With": "XMLHttpRequest",
                "Content-Length": "173"
            }
            body = {
                "curr_id": ID["CurrID"][t],
                "smlID": ID["smlID"][t],
                "header": i+"+Historical+Data",
                "st_date": std,
                "end_date": etd,
                "interval_sec": "Daily",
                "sort_col": "date",
                "sort_ord": "DESC",
                "action": "historical_data"
            }
            url = "https://www.investing.com/instruments/HistoricalDataAjax"
            re = requests.post(url, data=body, headers=headers)
            soup = BeautifulSoup(re.content, 'html.parser')
            table_rows = soup.findAll('tr')
            dates = []
            price = []
            open_price = []
            high = []
            low_price = []
            percentage_changes = []
            for tr in table_rows:
                metadata = tr.findAll('td', {'data-real-value': True})
                try:
                    dates.append(metadata[0].text.replace(",","/").replace("2019","19").replace("Jan","1/").replace("Feb","2/").replace("Mar","3/").replace("Apr","4/").replace("May","5/").replace("Jun","6/").replace("Jul","7/").replace("Aug","8/").replace("Sep","9/").replace("Oct","10/").replace("Nov","11/").replace("Dec","12/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9").replace(" ",""))
                    price.append(metadata[1].text.replace(",",""))
                    open_price.append(metadata[2].text.replace(",",""))
                    high.append(metadata[3].text.replace(",",""))
                    low_price.append(metadata[4].text.replace(",",""))
                except:
                    continue
                try:
                    percentage_changes.append(tr.findAll('td', {'class': 'redFont'})[1].text)
                except:
                    percentage_changes.append(tr.findAll('td', {'class': 'greenFont'})[1].text)
            tot = []
            tot.append(dates)#.replace(",","/").replace("2019","19").replace("Jan","1/").replace("Feb","2/").replace("Mar","3/").replace("Apr","4/").replace("May","5/").replace("Jun","6/").replace("Jul","7/").replace("Aug","8/").replace("Sep","9/").replace("Oct","10/").replace("Nov","11/").replace("Dec","12/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
            tot.append(open_price)
            tot.append(price)
            tot.append(high)
            tot.append(low_price)
            tot.append(percentage_changes)
            df1 = pd.DataFrame(tot)
            df = df1.T
            df.columns=[i+" Dates", i+" Close", i+" Open", i+" High", i+" Low", i+" % Change"]
#             df.columns=["Dates", i]
            df.index= df[i+" Dates"]
#             del df["Dates"]
            return(df) 
# for i in investing("AUD/CAD", "03/27/2019", "07/18/2019")["Dates"]:
#     print(type(i))
investing("BTC/EUR", "03/27/2019", "07/18/2019")

,BTC/EUR Dates,BTC/EUR Close,BTC/EUR Open,BTC/EUR High,BTC/EUR Low,BTC/EUR % Change
BTC/EUR Dates,,,,,,
7/18/19,7/18/19,8643.3,9450.0,9579.0,8301.0,9.33%
7/17/19,7/17/19,8396.8,8643.3,8918.7,8102.0,2.94%
7/16/19,7/16/19,9653.7,8396.8,9808.1,8369.1,-13.02%
7/15/19,7/15/19,9061.0,9653.7,9850.5,8780.0,6.54%
7/14/19,7/14/19,10120.4,9061.0,10179.0,8999.0,-10.47%
7/13/19,7/13/19,10486.6,10120.4,10518.3,9608.2,-3.49%
7/12/19,7/12/19,10100.0,10486.6,10600.0,9852.0,3.83%
7/11/19,7/11/19,10768.3,10100.0,10768.3,9777.7,-6.21%
7/10/19,7/10/19,11216.9,10768.4,11700.0,10271.1,-4.00%


In [4]:
listcurr=[]
for i in listcurr1:
    if "BTC" not in i:
        listcurr.append(i)
listcurr

['AUD/CAD',
 'AUD/CHF',
 'AUD/EUR',
 'AUD/GBP',
 'AUD/JPY',
 'AUD/NZD',
 'AUD/USD',
 'CAD/AUD',
 'CAD/CHF',
 'CAD/EUR',
 'CAD/JPY',
 'CHF/AUD',
 'CHF/EUR',
 'CHF/GBP',
 'CHF/JPY',
 'EUR/AUD',
 'EUR/CAD',
 'EUR/DKK',
 'EUR/GBP',
 'EUR/JPY',
 'EUR/NOK',
 'EUR/NZD',
 'EUR/RUB',
 'EUR/SEK',
 'EUR/TRY',
 'EUR/USD',
 'EUR/ZAR',
 'GBP/AUD',
 'GBP/CAD',
 'GBP/CHF',
 'GBP/DKK',
 'GBP/EUR',
 'GBP/JPY',
 'GBP/NZD',
 'GBP/USD',
 'JPY/CHF',
 'JPY/CNY',
 'JPY/EUR',
 'JPY/GBP',
 'NOK/JPY',
 'NOK/SEK',
 'NZD/JPY',
 'NZD/USD',
 'RUB/GBP',
 'SEK/JPY',
 'USD/ARS',
 'USD/BRL',
 'USD/CAD',
 'USD/CHF',
 'USD/CNY',
 'USD/CZK',
 'USD/DKK',
 'USD/EGP',
 'USD/HKD',
 'USD/HUF',
 'USD/IDR',
 'USD/ILS',
 'USD/INR',
 'USD/JPY',
 'USD/KRW',
 'USD/MXN',
 'USD/MYR',
 'USD/NOK',
 'USD/PLN',
 'USD/RUB',
 'USD/SEK',
 'USD/SGD',
 'USD/THB',
 'USD/TRY',
 'USD/UAH',
 'USD/ZAR']

In [5]:
# ii = 'USD/NOK'
std1 = "03/26/2019"
etd1 = "07/28/2019"
# investing(ii,std1,etd1)
combopen = []
combclose = []
combhigh = []
comblow = []
combdate = []
for ii in listcurr:
    datfr = investing(ii,std1,etd1)
    combopen.append(datfr[ii + " Open"] )
    combclose.append(datfr[ii + " Close"] )
    combhigh.append(datfr[ii + " High"] )
    comblow.append(datfr[ii + " Low"] )
    combdate.append(datfr[ii +" Dates"] )
Open = pd.concat(combopen,axis=1,sort=True)
Close = pd.concat(combclose,axis=1,sort=True)
High = pd.concat(combhigh,axis=1,sort=True)
Low = pd.concat(comblow,axis=1,sort=True)
DAte = pd.concat(combdate,axis=1,sort=True)

In [6]:
colopen=[]
for i in Open.columns:
    colopen.append(i.replace(" Open", ""))
Open.columns=colopen# Dataframe of open prices
for y in Open.columns:
    Open[y].fillna( method ='ffill', inplace = True)
colclose=[]
for i in Close.columns:
    colclose.append(i.replace(" Close", ""))
Close.columns=colclose# Dataframe of close prices
for y in Close.columns:
    Close[y].fillna( method ='ffill', inplace = True)
colhigh=[]
for i in High.columns:
    colhigh.append(i.replace(" High", ""))
High.columns=colhigh# Dataframe of high prices
for y in High.columns:
    High[y].fillna( method ='ffill', inplace = True)
collow=[]
for i in Low.columns:
    collow.append(i.replace(" Low", ""))
Low.columns=collow# Dataframe of low prices
for y in Low.columns:
    Low[y].fillna( method ='ffill', inplace = True)
# print(Low["BTC/EUR"]["7/28/19"])
# print(High["BTC/EUR"]["7/28/19"])
# print(Close["BTC/EUR"]["7/28/19"])
# print(Open["BTC/EUR"]["7/28/19"])
print(Low)
print(High)
print(Close)
print(Open)

        AUD/CAD AUD/CHF AUD/EUR AUD/GBP AUD/JPY AUD/NZD AUD/USD CAD/AUD  \
3/26/19  0.9520  0.7049  0.6274  0.5374   78.11  1.0276  0.7105  1.0451   
3/27/19  0.9486  0.7028  0.6280  0.5336   77.99  1.0306  0.7066  1.0458   
3/28/19  0.9482  0.7034  0.6287  0.5359   77.91  1.0399  0.7063  1.0488   
3/29/19  0.9463  0.7036  0.6297  0.5395   78.26  1.0400  0.7073  1.0483   
3/30/19  0.9463  0.7036  0.6297  0.5395   78.26  1.0400  0.7073  1.0483   
3/31/19  0.9481  0.7070  0.6329  0.5452   78.81  1.0418  0.7104  1.0504   
4/1/19   0.9457  0.7067  0.6324  0.5410   78.69  1.0413  0.7101  1.0495   
4/10/19  0.9479  0.7112  0.6313  0.5445   79.00  1.0541  0.7109  1.0462   
4/11/19  0.9517  0.7140  0.6319  0.5444   79.40  1.0577  0.7114  1.0449   
4/12/19  0.9516  0.7135  0.6307  0.5447   79.43  1.0578  0.7115  1.0430   
4/13/19  0.9516  0.7135  0.6307  0.5447   79.43  1.0578  0.7115  1.0430   
4/14/19  0.9554  0.7187  0.6342  0.5480   80.29  1.0586  0.7169  1.0439   
4/15/19  0.9539  0.7174  

In [7]:
BE = investing("BTC/EUR", "03/26/2019", "07/28/2019")
UB = investing("USD/BTC", "03/26/2019", "07/28/2019") # make sure the dates are The same

Low["BTC/EUR"] = BE["BTC/EUR Low"]
High["BTC/EUR"] = BE["BTC/EUR High"]
Close["BTC/EUR"] = BE["BTC/EUR Close"]
Open["BTC/EUR"] = BE["BTC/EUR Open"]
Low["USD/BTC"] = UB["USD/BTC Low"]
High["USD/BTC"] = UB["USD/BTC High"]
Close["USD/BTC"] = UB["USD/BTC Close"]
Open["USD/BTC"] = UB["USD/BTC Open"]

In [8]:
# This code gets the file names of all CSVs of interests
filepath = str(input("Entire file path of CSVs: " + "\n"))
def computer(filepath):
    if filepath[2] == '\\':
        filep = filepath[2:]
        path_for_windows = filep
        path = path_for_windows.replace('\\','/')
        return path
    else:
        return filepath
p = computer(filepath)
onlyfiles = [f for f in listdir(p) if isfile(join(p, f))]
final = []
for i in onlyfiles:
    final.append(p +"/"+ i)
print(final) #C:\Users\clare\OneDrive\Documents\Internships and work\IKNOWFIRST\IKF WORK\project\Portfolio\Dat

Entire file path of CSVs: 
C:\Users\clare\OneDrive\Documents\Internships and work\IKNOWFIRST\IKF WORK\project\Portfolio\Dat
['/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_01_May_2019.csv', '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_02_Apr_2019.csv', '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_02_Jul_2019.csv', '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_02_Jun_2019.csv', '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_t

In [9]:
DATA=[]
specific =[]
for u in final:
# u = 'C:/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_01_May_2019.csv' 
    actual = u[-15:]
    datee = (u[-15:].replace("_","/").replace(".csv","").replace("2019","19").replace("May","5").replace("Mar","3").replace("Apr","4").replace("Jun","6").replace("Jul","7").replace("Aug","8")
)
    date = datee.replace("0","")
    dateee=(u[-15:].replace("_","/").replace(".csv","").replace("May","05").replace("Mar","03").replace("Apr","04").replace("Jun","06").replace("Jul","07").replace("Aug","08")
)
    datetimeobject = datetime.strptime(dateee,'%d/%m/%Y') # to change from dd/mm/yyyy to a datetime object
    final_date = datetimeobject.strftime('%m/%d/%Y')# to change to mm/dd/yyyy
    final_date = final_date.replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9")
    df = pd.read_csv(u)
    for i in range (len(df)):
        for curr in listcurr:
            if df['Ticker'][i] == curr and df['Time Horizon'][i]=="1d":
                DATA.append([final_date, df['Ticker'][i],df['Time Horizon'][i],df['Predictability'][i],df['Signal'][i]])
                specific.append([dateee, df['Ticker'][i],df['Time Horizon'][i],df['Predictability'][i],df['Signal'][i]])         
data_top_10 = pd.DataFrame(DATA)
data_top_10.columns=(["Date","Currency","Time Horizon","Predictability","Signal"])
data_top_10.index = data_top_10["Date"]
# t = pd.DataFrame(DATA)
# t.columns=(["Date","Currency","Time Horizon","Predictability","Signal"])
# t.index = t["Date"]
data_top_10

,Date,Currency,Time Horizon,Predictability,Signal
Date,,,,,
5/1/19,5/1/19,GBP/AUD,1d,0.14,7.712283
5/1/19,5/1/19,USD/IDR,1d,0.12,7.277458
5/1/19,5/1/19,EUR/RUB,1d,0.13,6.607806
5/1/19,5/1/19,GBP/USD,1d,0.10,5.623930
5/1/19,5/1/19,GBP/EUR,1d,0.10,3.956948
5/1/19,5/1/19,JPY/CNY,1d,0.15,3.920768
5/1/19,5/1/19,CHF/JPY,1d,0.10,-3.497643
5/1/19,5/1/19,EUR/GBP,1d,0.11,-3.961699
5/1/19,5/1/19,AUD/GBP,1d,0.16,-8.279260


In [125]:
# Helper function 
def fivemax(ccsv):
    list1 = [abs(number) for number in list(ccsv['Signal'])]
    final_list = [] 
    for i in range(0, 5):  
        max1 = 0
        for j in range(len(list1)):      
            if list1[j] > max1: 
                max1 = list1[j]; 
        list1.remove(max1); 
        final_list.append(str(max1)) 
    return(final_list) 

def inverseBP (xxx): 
    return(xxx[-3:]+"/"+xxx[:3])
def repeatpairs(BBPP):
    if len(BBPP) == 2:
        P1=([BBPP[0],inverseBP(BBPP[0])])
        return(P1)
    elif len(BBPP) == 4:
        P1=[BBPP[0],inverseBP(BBPP[0])]
        P2=[BBPP[1],inverseBP(BBPP[1])]
        return(P1,P2)
    elif len(BBPP) == 6:
        P1=([BBPP[0],inverseBP(BBPP[0])])
        P2=([BBPP[1],inverseBP(BBPP[1])])
        P3=([BBPP[2],inverseBP(BBPP[2])])
        return(P1,P2,P3)
    elif len(BBPP) == 8:
        P1=([BBPP[0],inverseBP(BBPP[0])])
        P2=([BBPP[1],inverseBP(BBPP[1])])
        P3=([BBPP[2],inverseBP(BBPP[2])])
        P4=([BBPP[3],inverseBP(BBPP[3])])
        return(P1,P2,P3,P4)
    else:
        P1=([BBPP[0],inverseBP(BBPP[0])])
        P2=([BBPP[1],inverseBP(BBPP[1])])
        P3=([BBPP[2],inverseBP(BBPP[2])])
        P4=([BBPP[3],inverseBP(BBPP[3])])
        P5=([BBPP[4],inverseBP(BBPP[4])])
        return(P1,P2,P3,P4,P5)
def getmin(lismag): 
    if len(lismag) == 2:
        return(min(lismag))
    elif len(lismag) == 4:
        return (min([lismag[0],lismag[1]]),min([lismag[2],lismag[3]]))
    elif len(lismag) == 6:
        return (min([lismag[0],lismag[1]]),min([lismag[2],lismag[3]]),min([lismag[4],lismag[5]]))
    elif len(lismag) == 8:
        return (min([lismag[0],lismag[1]]),min([lismag[2],lismag[3]]),min([lismag[4],lismag[5]]),min([lismag[6],lismag[7]]))
    else:
        return (min([lismag[0],lismag[1]]),min([lismag[2],lismag[3]]),min([lismag[4],lismag[5]]),min([lismag[6],lismag[7]]),min([lismag[8],lismag[9]]))

# print(getmin([7.306373196827834, 7.819301495266081, 6.538918541888799, 7.7725400084841985]))
# print(getmin([]))    
def getunique(yyy):
    tix=[]
    for i in range(len(yyy)):
        tix.append(yyy["Ticker"][i])
    BPP=[]
    for i in tix:
        if inverseBP(i) in tix:
            BPP.append(i)
    lismagg=[]
    if len(BPP) >0:
        for uu in repeatpairs(BPP):
            for y in uu:
                for i in range(len(yyy)):
                    if yyy["Ticker"][i] == y:
                        lismagg.append(abs(float(yyy["Signal"][i])))
    indextorm =[]
    if len(lismagg) >0:
        for tt in getmin(lismagg):# to remove similar base pair from the top 10 dataframe
            for i in range(len(yyy)):
                if str(tt) in str(yyy["Signal"][i]):
                    indextorm.append(i)
    return(yyy.drop(yyy.index[indextorm])) 
import itertools
def gettopfivedf (csv11, T):
    csv = pd.read_csv(csv11)
    for1 = csv[0:10]
    for2 = csv[10:20]
    for3 = csv[20:30]
    for4 = csv[30:40]
    for5 = csv[40:50]
    for6 = csv[50:60]
    for2.index= list(range(len(for2)))
    for3.index= list(range(len(for3)))
    for4.index= list(range(len(for4)))
    for5.index= list(range(len(for5)))
    for6.index= list(range(len(for6)))
    f1 = getunique(for1)
    f2 = getunique(for2)
    f3 = getunique(for3)
    f4 = getunique(for4)
    f5 = getunique(for5)
    f6 = getunique(for6) 
    f1.index= list(range(len(f1)))
    f2.index= list(range(len(f2)))
    f3.index= list(range(len(f3)))
    f4.index= list(range(len(f4)))
    f5.index= list(range(len(f5)))
    f6.index= list(range(len(f6)))

    dlist = [f1,f2,f3,f4,f5,f6]
    futdf=[]
    for y in dlist:
        for r in fivemax(y):
            for i in range(len(y)):
                if r in str(y["Signal"][i]) and y['Time Horizon'][i]==T:
                    futdf.append([y['Ticker'][i],y['Time Horizon'][i],y['Predictability'][i],y['Signal'][i]])
    gf = pd.DataFrame(futdf, columns = ["Currency", "Time Horizon", "Predictability", "Signal"])
    dateee=(csv11[-15:].replace("_","/").replace(".csv","").replace("May","05").replace("Mar","03").replace("Apr","04").replace("Jun","06").replace("Jul","07").replace("Aug","08"))
    datetimeobject = datetime.strptime(dateee,'%d/%m/%Y') # to change from dd/mm/yyyy to a datetime object
    final_date = datetimeobject.strftime('%m/%d/%Y')# to change to mm/dd/yyyy
    final_date = final_date.replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9")
    L = list(itertools.repeat(final_date, len(gf)))
    gf["Date"] = L
    return(gf)
csv1 = '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_07_May_2019.csv'
# gettopfivedf (csv1)
listofdfs1d=[]
listofdfs2d=[]
listofdfs3d=[]
listofdfs4d=[]
listofdfs5d=[]
listofdfs6d=[]
for u in final:
    fin1 = gettopfivedf(u,"1d")
    fin2 = gettopfivedf(u,"2d")
    fin3 = gettopfivedf(u,"3d")
    fin4 = gettopfivedf(u,"4d")
    fin5 = gettopfivedf(u,"5d")
    fin6 = gettopfivedf(u,"6d")
    listofdfs1d.append(fin1)
    listofdfs2d.append(fin2)
    listofdfs3d.append(fin3)
    listofdfs4d.append(fin4)
    listofdfs5d.append(fin5)
    listofdfs6d.append(fin6)

data_top_5_1d = pd.concat(listofdfs1d)
data_top_5_2d = pd.concat(listofdfs2d)
data_top_5_3d = pd.concat(listofdfs3d)
data_top_5_4d = pd.concat(listofdfs4d)
data_top_5_5d = pd.concat(listofdfs5d)
data_top_5_6d = pd.concat(listofdfs6d)

data_top_5_1d.index = list(range(len(data_top_5_1d)))
data_top_5_2d.index = list(range(len(data_top_5_2d)))
data_top_5_3d.index = list(range(len(data_top_5_3d)))
data_top_5_4d.index = list(range(len(data_top_5_4d)))
data_top_5_5d.index = list(range(len(data_top_5_5d)))
data_top_5_6d.index = list(range(len(data_top_5_6d)))

data_top_5_1d = data_top_5_1d.applymap(str)
data_top_5_2d = data_top_5_2d.applymap(str)
data_top_5_3d = data_top_5_3d.applymap(str)
data_top_5_4d = data_top_5_4d.applymap(str)
data_top_5_5d = data_top_5_5d.applymap(str)
data_top_5_6d = data_top_5_6d.applymap(str)

data_top_5_1d.index = data_top_5_1d["Date"]
data_top_5_2d.index = data_top_5_2d["Date"]
data_top_5_3d.index = data_top_5_3d["Date"]
data_top_5_4d.index = data_top_5_4d["Date"]
data_top_5_5d.index = data_top_5_5d["Date"]
data_top_5_6d.index = data_top_5_6d["Date"]
def positivity(data_top_5):
    positive = []
    for i in range(len(data_top_5)):
        if float(data_top_5["Signal"][i])>0:
            positive.append([data_top_5["Date"][i],data_top_5["Currency"][i],data_top_5["Time Horizon"][i],data_top_5["Predictability"][i],data_top_5["Signal"][i]])
    dfpos = pd.DataFrame(positive,columns=["Date","Currency","Time Horizon","Predictability","Signal"])
    dfpos.index=dfpos["Date"]
    return(dfpos)

def negativity(data_top_5):
    negative=[]
    for i in range(len(data_top_5)):
        if float(data_top_5["Signal"][i])<0:
            negative.append([data_top_5["Date"][i],data_top_5["Currency"][i],data_top_5["Time Horizon"][i],data_top_5["Predictability"][i],data_top_5["Signal"][i]])
    dfneg = pd.DataFrame(negative,columns=["Date","Currency","Time Horizon","Predictability","Signal"])
    dfneg.index=dfneg["Date"]
    return(dfneg)
negativity(data_top_5_3d)

,Date,Currency,Time Horizon,Predictability,Signal
Date,,,,,
5/1/19,5/1/19,AUD/GBP,3d,0.2,-7.819301495266081
5/1/19,5/1/19,RUB/GBP,3d,0.2,-7.632553580244568
5/1/19,5/1/19,CHF/JPY,3d,0.13,-6.606659007211567
4/2/19,4/2/19,EUR/ZAR,3d,0.15,-13.505586940934545
4/2/19,4/2/19,USD/ZAR,3d,0.15,-11.872820570900574
7/2/19,7/2/19,USD/ARS,3d,0.22,-9.996818899966108
7/2/19,7/2/19,EUR/ZAR,3d,0.14,-4.344597536309991
7/2/19,7/2/19,GBP/NZD,3d,0.21,-2.785597295449314
7/2/19,7/2/19,USD/ZAR,3d,0.14,-2.6704174550636157


In [127]:
# helper function
import datetime
def addbusdays(from_date, add_days):
    business_days_to_add = add_days
    x = from_date #"5/11/19"
    splt = x.replace("/","\n").split()
    current_date = datetime.date(int("20"+splt[2]), int(splt[0]), int(splt[1]))
    while business_days_to_add > 0:
        current_date += datetime.timedelta(days=1)
        weekday = current_date.weekday()
        if weekday >= 5: # sunday = 6
            continue
        business_days_to_add -= 1
    datE=current_date.strftime("%m/%d/%Y")
    date = datE.replace("/0","/").replace("2019","19").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9")
    return(date)
addbusdays("7/7/19",7)


'7/16/19'

In [102]:
def openopen (data_top_5):
    # Buy at Open, Sell at Open
    exit_1d = []
    exit_2d = []
    exit_3d = []
    exit_4d = []
    exit_5d = []
    exit_6d = []

    for i in range(len(data_top_5)):
        exit_1d.append(addbusdays(data_top_5["Date"][i],1))
        exit_2d.append(addbusdays(data_top_5["Date"][i],2))
        exit_3d.append(addbusdays(data_top_5["Date"][i],3))
        exit_4d.append(addbusdays(data_top_5["Date"][i],4))
        exit_5d.append(addbusdays(data_top_5["Date"][i],5))
        exit_6d.append(addbusdays(data_top_5["Date"][i],6))

    data_top_5["Exit 1d"] = exit_1d
    data_top_5["Exit 2d"] = exit_2d
    data_top_5["Exit 3d"] = exit_3d
    data_top_5["Exit 4d"] = exit_4d
    data_top_5["Exit 5d"] = exit_5d
    data_top_5["Exit 6d"] = exit_6d

    return1d=[]
    return2d=[]
    return3d=[]
    return4d=[]
    return5d=[]
    return6d=[]

    for i in range (len(data_top_5)):
        returns1d = (((float(Open[data_top_5["Currency"][i]][data_top_5["Exit 1d"][i]]) - float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns2d = (((float(Open[data_top_5["Currency"][i]][data_top_5["Exit 2d"][i]]) - float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns3d = (((float(Open[data_top_5["Currency"][i]][data_top_5["Exit 3d"][i]]) - float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns4d = (((float(Open[data_top_5["Currency"][i]][data_top_5["Exit 4d"][i]]) - float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns5d = (((float(Open[data_top_5["Currency"][i]][data_top_5["Exit 5d"][i]]) - float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns6d = (((float(Open[data_top_5["Currency"][i]][data_top_5["Exit 6d"][i]]) - float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        return1d.append(returns1d)
        return2d.append(returns2d)
        return3d.append(returns3d)
        return4d.append(returns4d)
        return5d.append(returns5d)
        return6d.append(returns6d)
    data_top_5["Returns 1d"] = return1d
    data_top_5["Returns 2d"] = return2d
    data_top_5["Returns 3d"] = return3d
    data_top_5["Returns 4d"] = return4d
    data_top_5["Returns 5d"] = return5d
    data_top_5["Returns 6d"] = return6d
    return(data_top_5)


In [103]:
def closeclose (data_top_5):
    # Buy at Open, Sell at Open
    exit_1d = []
    exit_2d = []
    exit_3d = []
    exit_4d = []
    exit_5d = []
    exit_6d = []

    for i in range(len(data_top_5)):
        exit_1d.append(addbusdays(data_top_5["Date"][i],1))
        exit_2d.append(addbusdays(data_top_5["Date"][i],2))
        exit_3d.append(addbusdays(data_top_5["Date"][i],3))
        exit_4d.append(addbusdays(data_top_5["Date"][i],4))
        exit_5d.append(addbusdays(data_top_5["Date"][i],5))
        exit_6d.append(addbusdays(data_top_5["Date"][i],6))

    data_top_5["Exit 1d"] = exit_1d
    data_top_5["Exit 2d"] = exit_2d
    data_top_5["Exit 3d"] = exit_3d
    data_top_5["Exit 4d"] = exit_4d
    data_top_5["Exit 5d"] = exit_5d
    data_top_5["Exit 6d"] = exit_6d

    return1d=[]
    return2d=[]
    return3d=[]
    return4d=[]
    return5d=[]
    return6d=[]

    for i in range (len(data_top_5)):
        returns1d = (((float(Close[data_top_5["Currency"][i]][data_top_5["Exit 1d"][i]]) - float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns2d = (((float(Close[data_top_5["Currency"][i]][data_top_5["Exit 2d"][i]]) - float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns3d = (((float(Close[data_top_5["Currency"][i]][data_top_5["Exit 3d"][i]]) - float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns4d = (((float(Close[data_top_5["Currency"][i]][data_top_5["Exit 4d"][i]]) - float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns5d = (((float(Close[data_top_5["Currency"][i]][data_top_5["Exit 5d"][i]]) - float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns6d = (((float(Close[data_top_5["Currency"][i]][data_top_5["Exit 6d"][i]]) - float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        return1d.append(returns1d)
        return2d.append(returns2d)
        return3d.append(returns3d)
        return4d.append(returns4d)
        return5d.append(returns5d)
        return6d.append(returns6d)
    data_top_5["Returns 1d"] = return1d
    data_top_5["Returns 2d"] = return2d
    data_top_5["Returns 3d"] = return3d
    data_top_5["Returns 4d"] = return4d
    data_top_5["Returns 5d"] = return5d
    data_top_5["Returns 6d"] = return6d
    return(data_top_5)


In [104]:
def openclose (data_top_5):
    # Buy at Open, Sell at Open
    exit_1d = []
    exit_2d = []
    exit_3d = []
    exit_4d = []
    exit_5d = []
    exit_6d = []

    for i in range(len(data_top_5)):
        exit_1d.append(addbusdays(data_top_5["Date"][i],1))
        exit_2d.append(addbusdays(data_top_5["Date"][i],2))
        exit_3d.append(addbusdays(data_top_5["Date"][i],3))
        exit_4d.append(addbusdays(data_top_5["Date"][i],4))
        exit_5d.append(addbusdays(data_top_5["Date"][i],5))
        exit_6d.append(addbusdays(data_top_5["Date"][i],6))

    data_top_5["Exit 1d"] = exit_1d
    data_top_5["Exit 2d"] = exit_2d
    data_top_5["Exit 3d"] = exit_3d
    data_top_5["Exit 4d"] = exit_4d
    data_top_5["Exit 5d"] = exit_5d
    data_top_5["Exit 6d"] = exit_6d

    return1d=[]
    return2d=[]
    return3d=[]
    return4d=[]
    return5d=[]
    return6d=[]

    for i in range (len(data_top_5)):
        returns1d = (((float(Close[data_top_5["Currency"][i]][data_top_5["Exit 1d"][i]]) - float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns2d = (((float(Close[data_top_5["Currency"][i]][data_top_5["Exit 2d"][i]]) - float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns3d = (((float(Close[data_top_5["Currency"][i]][data_top_5["Exit 3d"][i]]) - float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns4d = (((float(Close[data_top_5["Currency"][i]][data_top_5["Exit 4d"][i]]) - float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns5d = (((float(Close[data_top_5["Currency"][i]][data_top_5["Exit 5d"][i]]) - float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns6d = (((float(Close[data_top_5["Currency"][i]][data_top_5["Exit 6d"][i]]) - float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Open[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        return1d.append(returns1d)
        return2d.append(returns2d)
        return3d.append(returns3d)
        return4d.append(returns4d)
        return5d.append(returns5d)
        return6d.append(returns6d)
    data_top_5["Returns 1d"] = return1d
    data_top_5["Returns 2d"] = return2d
    data_top_5["Returns 3d"] = return3d
    data_top_5["Returns 4d"] = return4d
    data_top_5["Returns 5d"] = return5d
    data_top_5["Returns 6d"] = return6d
    return(data_top_5)


In [105]:
def closeopen (data_top_5):
    # Buy at Open, Sell at Open
    exit_1d = []
    exit_2d = []
    exit_3d = []
    exit_4d = []
    exit_5d = []
    exit_6d = []

    for i in range(len(data_top_5)):
        exit_1d.append(addbusdays(data_top_5["Date"][i],1))
        exit_2d.append(addbusdays(data_top_5["Date"][i],2))
        exit_3d.append(addbusdays(data_top_5["Date"][i],3))
        exit_4d.append(addbusdays(data_top_5["Date"][i],4))
        exit_5d.append(addbusdays(data_top_5["Date"][i],5))
        exit_6d.append(addbusdays(data_top_5["Date"][i],6))

    data_top_5["Exit 1d"] = exit_1d
    data_top_5["Exit 2d"] = exit_2d
    data_top_5["Exit 3d"] = exit_3d
    data_top_5["Exit 4d"] = exit_4d
    data_top_5["Exit 5d"] = exit_5d
    data_top_5["Exit 6d"] = exit_6d

    return1d=[]
    return2d=[]
    return3d=[]
    return4d=[]
    return5d=[]
    return6d=[]

    for i in range (len(data_top_5)):
        returns1d = (((float(Open[data_top_5["Currency"][i]][data_top_5["Exit 1d"][i]]) - float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns2d = (((float(Open[data_top_5["Currency"][i]][data_top_5["Exit 2d"][i]]) - float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns3d = (((float(Open[data_top_5["Currency"][i]][data_top_5["Exit 3d"][i]]) - float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns4d = (((float(Open[data_top_5["Currency"][i]][data_top_5["Exit 4d"][i]]) - float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns5d = (((float(Open[data_top_5["Currency"][i]][data_top_5["Exit 5d"][i]]) - float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns6d = (((float(Open[data_top_5["Currency"][i]][data_top_5["Exit 6d"][i]]) - float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Close[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        return1d.append(returns1d)
        return2d.append(returns2d)
        return3d.append(returns3d)
        return4d.append(returns4d)
        return5d.append(returns5d)
        return6d.append(returns6d)
    data_top_5["Returns 1d"] = return1d
    data_top_5["Returns 2d"] = return2d
    data_top_5["Returns 3d"] = return3d
    data_top_5["Returns 4d"] = return4d
    data_top_5["Returns 5d"] = return5d
    data_top_5["Returns 6d"] = return6d
    return(data_top_5)


In [106]:
def highhigh (data_top_5):
    # Buy at Open, Sell at Open
    exit_1d = []
    exit_2d = []
    exit_3d = []
    exit_4d = []
    exit_5d = []
    exit_6d = []

    for i in range(len(data_top_5)):
        exit_1d.append(addbusdays(data_top_5["Date"][i],1))
        exit_2d.append(addbusdays(data_top_5["Date"][i],2))
        exit_3d.append(addbusdays(data_top_5["Date"][i],3))
        exit_4d.append(addbusdays(data_top_5["Date"][i],4))
        exit_5d.append(addbusdays(data_top_5["Date"][i],5))
        exit_6d.append(addbusdays(data_top_5["Date"][i],6))

    data_top_5["Exit 1d"] = exit_1d
    data_top_5["Exit 2d"] = exit_2d
    data_top_5["Exit 3d"] = exit_3d
    data_top_5["Exit 4d"] = exit_4d
    data_top_5["Exit 5d"] = exit_5d
    data_top_5["Exit 6d"] = exit_6d

    return1d=[]
    return2d=[]
    return3d=[]
    return4d=[]
    return5d=[]
    return6d=[]

    for i in range (len(data_top_5)):
        returns1d = (((float(High[data_top_5["Currency"][i]][data_top_5["Exit 1d"][i]]) - float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns2d = (((float(High[data_top_5["Currency"][i]][data_top_5["Exit 2d"][i]]) - float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns3d = (((float(High[data_top_5["Currency"][i]][data_top_5["Exit 3d"][i]]) - float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns4d = (((float(High[data_top_5["Currency"][i]][data_top_5["Exit 4d"][i]]) - float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns5d = (((float(High[data_top_5["Currency"][i]][data_top_5["Exit 5d"][i]]) - float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns6d = (((float(High[data_top_5["Currency"][i]][data_top_5["Exit 6d"][i]]) - float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        return1d.append(returns1d)
        return2d.append(returns2d)
        return3d.append(returns3d)
        return4d.append(returns4d)
        return5d.append(returns5d)
        return6d.append(returns6d)
    data_top_5["Returns 1d"] = return1d
    data_top_5["Returns 2d"] = return2d
    data_top_5["Returns 3d"] = return3d
    data_top_5["Returns 4d"] = return4d
    data_top_5["Returns 5d"] = return5d
    data_top_5["Returns 6d"] = return6d
    return(data_top_5)


In [107]:
def highlow (data_top_5):
    # Buy at Open, Sell at Open
    exit_1d = []
    exit_2d = []
    exit_3d = []
    exit_4d = []
    exit_5d = []
    exit_6d = []

    for i in range(len(data_top_5)):
        exit_1d.append(addbusdays(data_top_5["Date"][i],1))
        exit_2d.append(addbusdays(data_top_5["Date"][i],2))
        exit_3d.append(addbusdays(data_top_5["Date"][i],3))
        exit_4d.append(addbusdays(data_top_5["Date"][i],4))
        exit_5d.append(addbusdays(data_top_5["Date"][i],5))
        exit_6d.append(addbusdays(data_top_5["Date"][i],6))

    data_top_5["Exit 1d"] = exit_1d
    data_top_5["Exit 2d"] = exit_2d
    data_top_5["Exit 3d"] = exit_3d
    data_top_5["Exit 4d"] = exit_4d
    data_top_5["Exit 5d"] = exit_5d
    data_top_5["Exit 6d"] = exit_6d

    return1d=[]
    return2d=[]
    return3d=[]
    return4d=[]
    return5d=[]
    return6d=[]

    for i in range (len(data_top_5)):
        returns1d = (((float(Low[data_top_5["Currency"][i]][data_top_5["Exit 1d"][i]]) - float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns2d = (((float(Low[data_top_5["Currency"][i]][data_top_5["Exit 2d"][i]]) - float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns3d = (((float(Low[data_top_5["Currency"][i]][data_top_5["Exit 3d"][i]]) - float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns4d = (((float(Low[data_top_5["Currency"][i]][data_top_5["Exit 4d"][i]]) - float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns5d = (((float(Low[data_top_5["Currency"][i]][data_top_5["Exit 5d"][i]]) - float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns6d = (((float(Low[data_top_5["Currency"][i]][data_top_5["Exit 6d"][i]]) - float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(High[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        return1d.append(returns1d)
        return2d.append(returns2d)
        return3d.append(returns3d)
        return4d.append(returns4d)
        return5d.append(returns5d)
        return6d.append(returns6d)
    data_top_5["Returns 1d"] = return1d
    data_top_5["Returns 2d"] = return2d
    data_top_5["Returns 3d"] = return3d
    data_top_5["Returns 4d"] = return4d
    data_top_5["Returns 5d"] = return5d
    data_top_5["Returns 6d"] = return6d
    return(data_top_5)

In [108]:
def lowhigh (data_top_5):
    # Buy at Open, Sell at Open
    exit_1d = []
    exit_2d = []
    exit_3d = []
    exit_4d = []
    exit_5d = []
    exit_6d = []

    for i in range(len(data_top_5)):
        exit_1d.append(addbusdays(data_top_5["Date"][i],1))
        exit_2d.append(addbusdays(data_top_5["Date"][i],2))
        exit_3d.append(addbusdays(data_top_5["Date"][i],3))
        exit_4d.append(addbusdays(data_top_5["Date"][i],4))
        exit_5d.append(addbusdays(data_top_5["Date"][i],5))
        exit_6d.append(addbusdays(data_top_5["Date"][i],6))

    data_top_5["Exit 1d"] = exit_1d
    data_top_5["Exit 2d"] = exit_2d
    data_top_5["Exit 3d"] = exit_3d
    data_top_5["Exit 4d"] = exit_4d
    data_top_5["Exit 5d"] = exit_5d
    data_top_5["Exit 6d"] = exit_6d

    return1d=[]
    return2d=[]
    return3d=[]
    return4d=[]
    return5d=[]
    return6d=[]

    for i in range (len(data_top_5)):
        returns1d = (((float(High[data_top_5["Currency"][i]][data_top_5["Exit 1d"][i]]) - float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns2d = (((float(High[data_top_5["Currency"][i]][data_top_5["Exit 2d"][i]]) - float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns3d = (((float(High[data_top_5["Currency"][i]][data_top_5["Exit 3d"][i]]) - float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns4d = (((float(High[data_top_5["Currency"][i]][data_top_5["Exit 4d"][i]]) - float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns5d = (((float(High[data_top_5["Currency"][i]][data_top_5["Exit 5d"][i]]) - float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns6d = (((float(High[data_top_5["Currency"][i]][data_top_5["Exit 6d"][i]]) - float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        return1d.append(returns1d)
        return2d.append(returns2d)
        return3d.append(returns3d)
        return4d.append(returns4d)
        return5d.append(returns5d)
        return6d.append(returns6d)
    data_top_5["Returns 1d"] = return1d
    data_top_5["Returns 2d"] = return2d
    data_top_5["Returns 3d"] = return3d
    data_top_5["Returns 4d"] = return4d
    data_top_5["Returns 5d"] = return5d
    data_top_5["Returns 6d"] = return6d
    return(data_top_5)


In [109]:
def lowlow (data_top_5):
    # Buy at Open, Sell at Open
    exit_1d = []
    exit_2d = []
    exit_3d = []
    exit_4d = []
    exit_5d = []
    exit_6d = []

    for i in range(len(data_top_5)):
        exit_1d.append(addbusdays(data_top_5["Date"][i],1))
        exit_2d.append(addbusdays(data_top_5["Date"][i],2))
        exit_3d.append(addbusdays(data_top_5["Date"][i],3))
        exit_4d.append(addbusdays(data_top_5["Date"][i],4))
        exit_5d.append(addbusdays(data_top_5["Date"][i],5))
        exit_6d.append(addbusdays(data_top_5["Date"][i],6))

    data_top_5["Exit 1d"] = exit_1d
    data_top_5["Exit 2d"] = exit_2d
    data_top_5["Exit 3d"] = exit_3d
    data_top_5["Exit 4d"] = exit_4d
    data_top_5["Exit 5d"] = exit_5d
    data_top_5["Exit 6d"] = exit_6d

    return1d=[]
    return2d=[]
    return3d=[]
    return4d=[]
    return5d=[]
    return6d=[]

    for i in range (len(data_top_5)):
        returns1d = (((float(Low[data_top_5["Currency"][i]][data_top_5["Exit 1d"][i]]) - float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns2d = (((float(Low[data_top_5["Currency"][i]][data_top_5["Exit 2d"][i]]) - float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns3d = (((float(Low[data_top_5["Currency"][i]][data_top_5["Exit 3d"][i]]) - float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns4d = (((float(Low[data_top_5["Currency"][i]][data_top_5["Exit 4d"][i]]) - float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns5d = (((float(Low[data_top_5["Currency"][i]][data_top_5["Exit 5d"][i]]) - float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        returns6d = (((float(Low[data_top_5["Currency"][i]][data_top_5["Exit 6d"][i]]) - float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))/float(Low[data_top_5["Currency"][i]][data_top_5["Date"][i]]))*100)
        return1d.append(returns1d)
        return2d.append(returns2d)
        return3d.append(returns3d)
        return4d.append(returns4d)
        return5d.append(returns5d)
        return6d.append(returns6d)
    data_top_5["Returns 1d"] = return1d
    data_top_5["Returns 2d"] = return2d
    data_top_5["Returns 3d"] = return3d
    data_top_5["Returns 4d"] = return4d
    data_top_5["Returns 5d"] = return5d
    data_top_5["Returns 6d"] = return6d
    return(data_top_5)


In [154]:
def returns(g,TH):
    alif = []
    alll = []
    for i in range(len(g)):
        if (float(g["Signal"][i]) > 0 and g[TH][i] > 0) or (float(g["Signal"][i]) < 0 and g[TH][i] < 0):
            alll.append(abs(g[TH][i]))
        else:
            if g[TH][i] < 0:
                alif.append(g[TH][i])
            else:
                alif.append(-1*g[TH][i])
    alif = [x for x in alif if str(x) != 'nan']
    alll = [x for x in alll if str(x) != 'nan'] 
    if len(alif) > 0:
        average_losses= sum(alif)/len(alif)
    else:
        average_losses= 0
    if len(alll) >0:
        average_gains= sum(alll)/len(alll)
    else:
        average_gains= 0
#     if len(alll) >0 and len(alif)>0:
#         average_returns = (sum(alif) +sum(alll))/(len(alif)+len(alll)) #Average returns just by following signal per currency pair
    portf_val = alif+alll
    portf_valmean = sum(portf_val)/len(portf_val)
    Sharpe_Ratio = portf_valmean / statistics.stdev(portf_val)
    A_Sharpe_Ratio = (252**0.5)*Sharpe_Ratio
    TiHo = TH[-2:]
    hit_ratio = (len(alll)/len(g))*100
    return([TiHo, Sharpe_Ratio, A_Sharpe_Ratio, portf_valmean,hit_ratio])


In [155]:
data_top_5_1d_n = negativity(data_top_5_1d)
data_top_5_2d_n = negativity(data_top_5_2d)
data_top_5_3d_n = negativity(data_top_5_3d)
data_top_5_4d_n = negativity(data_top_5_4d)
data_top_5_5d_n = negativity(data_top_5_5d)
data_top_5_6d_n = negativity(data_top_5_6d)

data_top_5_1d_p = positivity(data_top_5_1d)
data_top_5_2d_p = positivity(data_top_5_2d)
data_top_5_3d_p = positivity(data_top_5_3d)
data_top_5_4d_p = positivity(data_top_5_4d)
data_top_5_5d_p = positivity(data_top_5_5d)
data_top_5_6d_p = positivity(data_top_5_6d)

data_top_5_1d_n

,Date,Currency,Time Horizon,Predictability,Signal
Date,,,,,
5/1/19,5/1/19,RUB/GBP,1d,0.15,-10.775369760345272
5/1/19,5/1/19,AUD/GBP,1d,0.16,-8.27926040675232
4/2/19,4/2/19,EUR/ZAR,1d,0.11,-14.586033896209312
4/2/19,4/2/19,USD/ZAR,1d,0.11,-12.647134955959313
4/2/19,4/2/19,USD/ARS,1d,0.11,-4.584187639458416
4/2/19,4/2/19,GBP/CAD,1d,0.13,-4.291760658867741
7/2/19,7/2/19,USD/ARS,1d,0.18,-5.80767574188507
7/2/19,7/2/19,EUR/ZAR,1d,0.1,-3.4352631682451102
7/2/19,7/2/19,JPY/CNY,1d,0.16,-2.2615840081286995


In [156]:
lowhigh(data_top_5_1d_n)
returns(lowhigh(data_top_5_1d_n), "Returns 1d")

['1d',
 -1.0956321246722407,
 -17.392620781776383,
 -0.8497250743454422,
 0.9009009009009009]

In [157]:
dfdf_p=[]
dfdf_p.append(returns(openopen(data_top_5_1d_p), "Returns 1d"))
dfdf_p.append(returns(openopen(data_top_5_2d_p), "Returns 2d"))
dfdf_p.append(returns(openopen(data_top_5_3d_p), "Returns 3d"))
dfdf_p.append(returns(openopen(data_top_5_4d_p), "Returns 4d"))
dfdf_p.append(returns(openopen(data_top_5_5d_p), "Returns 5d"))
dfdf_p.append(returns(openopen(data_top_5_6d_p), "Returns 6d"))
dfdf_p.append(returns(openclose(data_top_5_1d_p), "Returns 1d"))
dfdf_p.append(returns(openclose(data_top_5_2d_p), "Returns 2d"))
dfdf_p.append(returns(openclose(data_top_5_3d_p), "Returns 3d"))
dfdf_p.append(returns(openclose(data_top_5_4d_p), "Returns 4d"))
dfdf_p.append(returns(openclose(data_top_5_5d_p), "Returns 5d"))
dfdf_p.append(returns(openclose(data_top_5_6d_p), "Returns 6d"))
dfdf_p.append(returns(closeopen(data_top_5_1d_p), "Returns 1d"))
dfdf_p.append(returns(closeopen(data_top_5_2d_p), "Returns 2d"))
dfdf_p.append(returns(closeopen(data_top_5_3d_p), "Returns 3d"))
dfdf_p.append(returns(closeopen(data_top_5_4d_p), "Returns 4d"))
dfdf_p.append(returns(closeopen(data_top_5_5d_p), "Returns 5d"))
dfdf_p.append(returns(closeopen(data_top_5_6d_p), "Returns 6d"))
dfdf_p.append(returns(closeclose(data_top_5_1d_p), "Returns 1d"))
dfdf_p.append(returns(closeclose(data_top_5_2d_p), "Returns 2d"))
dfdf_p.append(returns(closeclose(data_top_5_3d_p), "Returns 3d"))
dfdf_p.append(returns(closeclose(data_top_5_4d_p), "Returns 4d"))
dfdf_p.append(returns(closeclose(data_top_5_5d_p), "Returns 5d"))
dfdf_p.append(returns(closeclose(data_top_5_6d_p), "Returns 6d"))
dfdf_p.append(returns(highhigh(data_top_5_1d_p), "Returns 1d"))
dfdf_p.append(returns(highhigh(data_top_5_2d_p), "Returns 2d"))
dfdf_p.append(returns(highhigh(data_top_5_3d_p), "Returns 3d"))
dfdf_p.append(returns(highhigh(data_top_5_4d_p), "Returns 4d"))
dfdf_p.append(returns(highhigh(data_top_5_5d_p), "Returns 5d"))
dfdf_p.append(returns(highhigh(data_top_5_6d_p), "Returns 6d"))
dfdf_p.append(returns(highlow(data_top_5_1d_p), "Returns 1d"))
dfdf_p.append(returns(highlow(data_top_5_2d_p), "Returns 2d"))
dfdf_p.append(returns(highlow(data_top_5_3d_p), "Returns 3d"))
dfdf_p.append(returns(highlow(data_top_5_4d_p), "Returns 4d"))
dfdf_p.append(returns(highlow(data_top_5_5d_p), "Returns 5d"))
dfdf_p.append(returns(highlow(data_top_5_6d_p), "Returns 6d"))
dfdf_p.append(returns(lowhigh(data_top_5_1d_p), "Returns 1d"))
dfdf_p.append(returns(lowhigh(data_top_5_2d_p), "Returns 2d"))
dfdf_p.append(returns(lowhigh(data_top_5_3d_p), "Returns 3d"))
dfdf_p.append(returns(lowhigh(data_top_5_4d_p), "Returns 4d"))
dfdf_p.append(returns(lowhigh(data_top_5_5d_p), "Returns 5d"))
dfdf_p.append(returns(lowhigh(data_top_5_6d_p), "Returns 6d"))
dfdf_p.append(returns(lowlow(data_top_5_1d_p), "Returns 1d"))
dfdf_p.append(returns(lowlow(data_top_5_2d_p), "Returns 2d"))
dfdf_p.append(returns(lowlow(data_top_5_3d_p), "Returns 3d"))
dfdf_p.append(returns(lowlow(data_top_5_4d_p), "Returns 4d"))
dfdf_p.append(returns(lowlow(data_top_5_5d_p), "Returns 5d"))
dfdf_p.append(returns(lowlow(data_top_5_6d_p), "Returns 6d"))

In [158]:
dfdf_n=[]
dfdf_n.append(returns(openopen(data_top_5_1d_n), "Returns 1d"))
dfdf_n.append(returns(openopen(data_top_5_2d_n), "Returns 2d"))
dfdf_n.append(returns(openopen(data_top_5_3d_n), "Returns 3d"))
dfdf_n.append(returns(openopen(data_top_5_4d_n), "Returns 4d"))
dfdf_n.append(returns(openopen(data_top_5_5d_n), "Returns 5d"))
dfdf_n.append(returns(openopen(data_top_5_6d_n), "Returns 6d"))
dfdf_n.append(returns(openclose(data_top_5_1d_n), "Returns 1d"))
dfdf_n.append(returns(openclose(data_top_5_2d_n), "Returns 2d"))
dfdf_n.append(returns(openclose(data_top_5_3d_n), "Returns 3d"))
dfdf_n.append(returns(openclose(data_top_5_4d_n), "Returns 4d"))
dfdf_n.append(returns(openclose(data_top_5_5d_n), "Returns 5d"))
dfdf_n.append(returns(openclose(data_top_5_6d_n), "Returns 6d"))
dfdf_n.append(returns(closeopen(data_top_5_1d_n), "Returns 1d"))
dfdf_n.append(returns(closeopen(data_top_5_2d_n), "Returns 2d"))
dfdf_n.append(returns(closeopen(data_top_5_3d_n), "Returns 3d"))
dfdf_n.append(returns(closeopen(data_top_5_4d_n), "Returns 4d"))
dfdf_n.append(returns(closeopen(data_top_5_5d_n), "Returns 5d"))
dfdf_n.append(returns(closeopen(data_top_5_6d_n), "Returns 6d"))
dfdf_n.append(returns(closeclose(data_top_5_1d_n), "Returns 1d"))
dfdf_n.append(returns(closeclose(data_top_5_2d_n), "Returns 2d"))
dfdf_n.append(returns(closeclose(data_top_5_3d_n), "Returns 3d"))
dfdf_n.append(returns(closeclose(data_top_5_4d_n), "Returns 4d"))
dfdf_n.append(returns(closeclose(data_top_5_5d_n), "Returns 5d"))
dfdf_n.append(returns(closeclose(data_top_5_6d_n), "Returns 6d"))
dfdf_n.append(returns(highhigh(data_top_5_1d_n), "Returns 1d"))
dfdf_n.append(returns(highhigh(data_top_5_2d_n), "Returns 2d"))
dfdf_n.append(returns(highhigh(data_top_5_3d_n), "Returns 3d"))
dfdf_n.append(returns(highhigh(data_top_5_4d_n), "Returns 4d"))
dfdf_n.append(returns(highhigh(data_top_5_5d_n), "Returns 5d"))
dfdf_n.append(returns(highhigh(data_top_5_6d_n), "Returns 6d"))
dfdf_n.append(returns(highlow(data_top_5_1d_n), "Returns 1d"))
dfdf_n.append(returns(highlow(data_top_5_2d_n), "Returns 2d"))
dfdf_n.append(returns(highlow(data_top_5_3d_n), "Returns 3d"))
dfdf_n.append(returns(highlow(data_top_5_4d_n), "Returns 4d"))
dfdf_n.append(returns(highlow(data_top_5_5d_n), "Returns 5d"))
dfdf_n.append(returns(highlow(data_top_5_6d_n), "Returns 6d"))
dfdf_n.append(returns(lowhigh(data_top_5_1d_n), "Returns 1d"))
dfdf_n.append(returns(lowhigh(data_top_5_2d_n), "Returns 2d"))
dfdf_n.append(returns(lowhigh(data_top_5_3d_n), "Returns 3d"))
dfdf_n.append(returns(lowhigh(data_top_5_4d_n), "Returns 4d"))
dfdf_n.append(returns(lowhigh(data_top_5_5d_n), "Returns 5d"))
dfdf_n.append(returns(lowhigh(data_top_5_6d_n), "Returns 6d"))
dfdf_n.append(returns(lowlow(data_top_5_1d_n), "Returns 1d"))
dfdf_n.append(returns(lowlow(data_top_5_2d_n), "Returns 2d"))
dfdf_n.append(returns(lowlow(data_top_5_3d_n), "Returns 3d"))
dfdf_n.append(returns(lowlow(data_top_5_4d_n), "Returns 4d"))
dfdf_n.append(returns(lowlow(data_top_5_5d_n), "Returns 5d"))
dfdf_n.append(returns(lowlow(data_top_5_6d_n), "Returns 6d"))

In [159]:
dfdf=[]
dfdf.append(returns(openopen(data_top_5_1d), "Returns 1d"))
dfdf.append(returns(openopen(data_top_5_2d), "Returns 2d"))
dfdf.append(returns(openopen(data_top_5_3d), "Returns 3d"))
dfdf.append(returns(openopen(data_top_5_4d), "Returns 4d"))
dfdf.append(returns(openopen(data_top_5_5d), "Returns 5d"))
dfdf.append(returns(openopen(data_top_5_6d), "Returns 6d"))
dfdf.append(returns(openclose(data_top_5_1d), "Returns 1d"))
dfdf.append(returns(openclose(data_top_5_2d), "Returns 2d"))
dfdf.append(returns(openclose(data_top_5_3d), "Returns 3d"))
dfdf.append(returns(openclose(data_top_5_4d), "Returns 4d"))
dfdf.append(returns(openclose(data_top_5_5d), "Returns 5d"))
dfdf.append(returns(openclose(data_top_5_6d), "Returns 6d"))
dfdf.append(returns(closeopen(data_top_5_1d), "Returns 1d"))
dfdf.append(returns(closeopen(data_top_5_2d), "Returns 2d"))
dfdf.append(returns(closeopen(data_top_5_3d), "Returns 3d"))
dfdf.append(returns(closeopen(data_top_5_4d), "Returns 4d"))
dfdf.append(returns(closeopen(data_top_5_5d), "Returns 5d"))
dfdf.append(returns(closeopen(data_top_5_6d), "Returns 6d"))
dfdf.append(returns(closeclose(data_top_5_1d), "Returns 1d"))
dfdf.append(returns(closeclose(data_top_5_2d), "Returns 2d"))
dfdf.append(returns(closeclose(data_top_5_3d), "Returns 3d"))
dfdf.append(returns(closeclose(data_top_5_4d), "Returns 4d"))
dfdf.append(returns(closeclose(data_top_5_5d), "Returns 5d"))
dfdf.append(returns(closeclose(data_top_5_6d), "Returns 6d"))
dfdf.append(returns(highhigh(data_top_5_1d), "Returns 1d"))
dfdf.append(returns(highhigh(data_top_5_2d), "Returns 2d"))
dfdf.append(returns(highhigh(data_top_5_3d), "Returns 3d"))
dfdf.append(returns(highhigh(data_top_5_4d), "Returns 4d"))
dfdf.append(returns(highhigh(data_top_5_5d), "Returns 5d"))
dfdf.append(returns(highhigh(data_top_5_6d), "Returns 6d"))
dfdf.append(returns(highlow(data_top_5_1d), "Returns 1d"))
dfdf.append(returns(highlow(data_top_5_2d), "Returns 2d"))
dfdf.append(returns(highlow(data_top_5_3d), "Returns 3d"))
dfdf.append(returns(highlow(data_top_5_4d), "Returns 4d"))
dfdf.append(returns(highlow(data_top_5_5d), "Returns 5d"))
dfdf.append(returns(highlow(data_top_5_6d), "Returns 6d"))
dfdf.append(returns(lowhigh(data_top_5_1d), "Returns 1d"))
dfdf.append(returns(lowhigh(data_top_5_2d), "Returns 2d"))
dfdf.append(returns(lowhigh(data_top_5_3d), "Returns 3d"))
dfdf.append(returns(lowhigh(data_top_5_4d), "Returns 4d"))
dfdf.append(returns(lowhigh(data_top_5_5d), "Returns 5d"))
dfdf.append(returns(lowhigh(data_top_5_6d), "Returns 6d"))
dfdf.append(returns(lowlow(data_top_5_1d), "Returns 1d"))
dfdf.append(returns(lowlow(data_top_5_2d), "Returns 2d"))
dfdf.append(returns(lowlow(data_top_5_3d), "Returns 3d"))
dfdf.append(returns(lowlow(data_top_5_4d), "Returns 4d"))
dfdf.append(returns(lowlow(data_top_5_5d), "Returns 5d"))
dfdf.append(returns(lowlow(data_top_5_6d), "Returns 6d"))

In [160]:
dfdfdf = pd.DataFrame(dfdf,columns=["Time Horizon","Sharpe Ratio", "Annualized Sharpe Ratio","IKF Returns", "IKF Signal Hit Ratio"])
dfdfdf["Strategy"] = ["Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low"]

dfdfdf.to_excel(r'C:\Users\clare\OneDrive\Documents\Internships and work\IKNOWFIRST\IKF WORK\project\Portfolio\Analysis\Various Strategies.xlsx',index=None, header = True)
dfdfdf

,Time Horizon,Sharpe Ratio,Annualized Sharpe Ratio,IKF Returns,IKF Signal Hit Ratio,Strategy
0,1d,0.082365,1.307497,0.129942,48.571429,"Buy at open , sell at open"
1,2d,0.061258,0.972448,0.087982,52.727273,"Buy at open , sell at open"
2,3d,0.077401,1.228705,0.135280,55.844156,"Buy at open , sell at open"
3,4d,0.102740,1.630942,0.183509,60.259740,"Buy at open , sell at open"
4,5d,0.066481,1.055352,0.139204,56.623377,"Buy at open , sell at open"
5,6d,0.068493,1.087298,0.167232,56.623377,"Buy at open , sell at open"
6,1d,0.026809,0.425572,0.018246,37.402597,"Buy at open , sell at close"
7,2d,0.036643,0.581694,0.027987,51.688312,"Buy at open , sell at close"
8,3d,0.055898,0.887354,0.091116,53.246753,"Buy at open , sell at close"
9,4d,0.063585,1.009381,0.109723,56.103896,"Buy at open , sell at close"


In [162]:
dfdfdf_n = pd.DataFrame(dfdf_n,columns=["Time Horizon","Sharpe Ratio", "Annualized Sharpe Ratio","IKF Returns", "IKF Signal Hit Ratio"])
dfdfdf_n["Strategy"] = ["Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low"]

dfdfdf_n.to_excel(r'C:\Users\clare\OneDrive\Documents\Internships and work\IKNOWFIRST\IKF WORK\project\Portfolio\Analysis\Various Strategies (Negative Signal).xlsx',index=None, header = True)
dfdfdf_n

,Time Horizon,Sharpe Ratio,Annualized Sharpe Ratio,IKF Returns,IKF Signal Hit Ratio,Strategy
0,1d,0.030737,0.487941,0.022021,48.198198,"Buy at open , sell at open"
1,2d,0.148518,2.357658,0.127740,56.696429,"Buy at open , sell at open"
2,3d,0.154864,2.458396,0.162722,58.407080,"Buy at open , sell at open"
3,4d,0.154646,2.454937,0.196390,62.608696,"Buy at open , sell at open"
4,5d,0.117347,1.862827,0.177813,59.656652,"Buy at open , sell at open"
5,6d,0.126203,2.003414,0.205871,59.641256,"Buy at open , sell at open"
6,1d,0.070658,1.121660,0.032386,45.045045,"Buy at open , sell at close"
7,2d,0.111980,1.777634,0.078162,54.910714,"Buy at open , sell at close"
8,3d,0.155353,2.466148,0.136583,56.194690,"Buy at open , sell at close"
9,4d,0.141691,2.249268,0.146412,58.695652,"Buy at open , sell at close"


In [163]:
dfdfdf_p = pd.DataFrame(dfdf_p,columns=["Time Horizon","Sharpe Ratio", "Annualized Sharpe Ratio","IKF Returns", "IKF Signal Hit Ratio"])
dfdfdf_p["Strategy"] = ["Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at open",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at open , sell at close",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at open",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at close , sell at close",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at high",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at high , sell at low",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at high",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low",
                      "Buy at Low , sell at low"]

dfdfdf_p.to_excel(r'C:\Users\clare\OneDrive\Documents\Internships and work\IKNOWFIRST\IKF WORK\project\Portfolio\Analysis\Various Strategies (Positive Signal).xlsx',index=None, header = True)
dfdfdf_p

,Time Horizon,Sharpe Ratio,Annualized Sharpe Ratio,IKF Returns,IKF Signal Hit Ratio,Strategy
0,1d,0.121888,1.934912,0.276926,49.079755,"Buy at open , sell at open"
1,2d,0.016511,0.262108,0.032667,47.204969,"Buy at open , sell at open"
2,3d,0.039807,0.631915,0.096274,52.201258,"Buy at open , sell at open"
3,4d,0.069742,1.107115,0.164396,56.774194,"Buy at open , sell at open"
4,5d,0.028996,0.460293,0.080021,51.973684,"Buy at open , sell at open"
5,6d,0.035121,0.557526,0.114044,52.469136,"Buy at open , sell at open"
6,1d,-0.001123,-0.017829,-0.001011,26.993865,"Buy at open , sell at close"
7,2d,-0.049546,-0.786516,-0.041822,47.204969,"Buy at open , sell at close"
8,3d,0.011453,0.181814,0.026491,49.056604,"Buy at open , sell at close"
9,4d,0.022891,0.363384,0.055282,52.258065,"Buy at open , sell at close"
